In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import datetime
from IPython.display import clear_output
import lightgbm as lgb
from bayes_opt import BayesianOptimization


RANDOM_STATE = 42

In [2]:
from generate_features import generate_features
from read_utils import read_data

In [8]:
from settings import CATEGORICAL_FEATURES, COLUMNS_TO_REMOVE

In [3]:
train, test, sample_submission = read_data()

In [4]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [5]:
%%time
train_test_transformed = generate_features(train, test)

/home/ec2-user/FRAUD/IEEE_FRAUD/feature_engineering.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lambda x: x if ' ' not in x else ' '.join(x.split()[:-1])
/home/ec2-user/FRAUD/IEEE_FRAUD/feature_engineering.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lambda x: parse_version(x.split()[-1])[0]
/home/ec2-user/FRAUD/IEEE_FRAUD/feature_engineering.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lambda x: parse_version(

mean
       count      mean       std
addr1                           
100.0     26  0.000000  0.000000
101.0      1  0.000000       NaN
102.0     13  0.000000  0.000000
104.0      8  0.000000  0.000000
105.0      3  0.000000  0.000000
...      ...       ...       ...
530.0      1  0.000000       NaN
531.0      1  0.000000       NaN
535.0      2  0.000000  0.000000
536.0    282  0.113475  0.317737
540.0      6  0.000000  0.000000

[330 rows x 3 columns]
std
       count      mean       std
addr1                           
100.0     26  0.000000  0.000000
101.0      1  0.000000       NaN
102.0     13  0.000000  0.000000
104.0      8  0.000000  0.000000
105.0      3  0.000000  0.000000
...      ...       ...       ...
530.0      1  0.000000       NaN
531.0      1  0.000000       NaN
535.0      2  0.000000  0.000000
536.0    282  0.113475  0.317737
540.0      6  0.000000  0.000000

[330 rows x 3 columns]
mean
       count      mean       std
addr2                           
10.0       8  

mean
                            count      mean       std
TransactionDT_weekOfMonth                            
1                          124450  0.034239  0.181842
2                          110472  0.035394  0.184773
3                          115196  0.034255  0.181883
4                          115773  0.034516  0.182551
5                           35323  0.040653  0.197489
std
                            count      mean       std
TransactionDT_weekOfMonth                            
1                          124450  0.034239  0.181842
2                          110472  0.035394  0.184773
3                          115196  0.034255  0.181883
4                          115773  0.034516  0.182551
5                           35323  0.040653  0.197489
mean
            count      mean       std
DeviceType                           
desktop     74625  0.063611  0.244061
mobile      47872  0.098450  0.297925
std
            count      mean       std
DeviceType                          

mean
                          count        mean         std  nanmedian
TransactionDT_dayOfWeek                                           
0                        162578  136.137457  247.457166    68.5000
1                        161510  139.141977  249.086911    67.9500
2                        158672  141.983459  278.590694    71.3255
3                        157223  140.864879  251.321189    70.0000
4                        179744  146.530553  264.386266    75.0000
5                        146223  124.187522  200.753522    66.3520
6                        131281  108.350388  164.607740    59.0000
std
                          count        mean         std  nanmedian
TransactionDT_dayOfWeek                                           
0                        162578  136.137457  247.457166    68.5000
1                        161510  139.141977  249.086911    67.9500
2                        158672  141.983459  278.590694    71.3255
3                        157223  140.864879  251.3211

mean
                            count        mean         std  nanmedian
TransactionDT_weekOfMonth                                           
1                          266208  135.935386  239.270201     69.319
2                          240264  133.849335  252.807296     67.950
3                          255763  134.138691  234.245081     68.950
4                          251073  133.211206  235.116871     67.950
5                           83923  141.837953  264.507583     67.950
std
                            count        mean         std  nanmedian
TransactionDT_weekOfMonth                                           
1                          266208  135.935386  239.270201     69.319
2                          240264  133.849335  252.807296     67.950
3                          255763  134.138691  234.245081     68.950
4                          251073  133.211206  235.116871     67.950
5                           83923  141.837953  264.507583     67.950
<function nanmedian at 0x

...

mean
                          count        mean         std  nanmedian
TransactionDT_dayOfWeek                                           
0                        144482  197.525180  246.542759       63.0
1                        147984  189.403618  241.633866       55.0
2                        146676  187.718877  240.767488       53.0
3                        145176  180.739619  236.628903       45.0
4                        163617  176.873393  233.324895       42.0
5                        131747  178.719660  235.425221       42.0
6                        116367  185.575292  239.323509       51.0
std
                          count        mean         std  nanmedian
TransactionDT_dayOfWeek                                           
0                        144482  197.525180  246.542759       63.0
1                        147984  189.403618  241.633866       55.0
2                        146676  187.718877  240.767488       53.0
3                        145176  180.739619  236.6289

mean
                            count        mean         std  nanmedian
TransactionDT_weekOfMonth                                           
1                          244275  193.473208  238.676308       67.0
2                          213842  185.652940  237.291865       54.0
3                          231033  181.741872  239.498809       43.0
4                          227411  178.221014  239.801971       34.0
5                           79488  187.974373  242.042953       53.0
std
                            count        mean         std  nanmedian
TransactionDT_weekOfMonth                                           
1                          244275  193.473208  238.676308       67.0
2                          213842  185.652940  237.291865       54.0
3                          231033  181.741872  239.498809       43.0
4                          227411  178.221014  239.801971       34.0
5                           79488  187.974373  242.042953       53.0
<function nanmedian at 0x

mean
       count      mean       std  nanmedian
addr2                                      
10.0      11  0.000000  0.000000        0.0
11.0       1  1.000000       NaN        1.0
12.0       4  0.000000  0.000000        0.0
13.0       8  1.625000  0.744024        1.5
14.0       1  1.000000       NaN        1.0
...      ...       ...       ...        ...
98.0      26  0.769231  0.710363        1.0
99.0       1  1.000000       NaN        1.0
100.0      7  0.571429  0.786796        0.0
101.0      9  1.111111  0.927961        1.0
102.0     21  1.142857  0.853564        1.0

[93 rows x 4 columns]
std
       count      mean       std  nanmedian
addr2                                      
10.0      11  0.000000  0.000000        0.0
11.0       1  1.000000       NaN        1.0
12.0       4  0.000000  0.000000        0.0
13.0       8  1.625000  0.744024        1.5
14.0       1  1.000000       NaN        1.0
...      ...       ...       ...        ...
98.0      26  0.769231  0.710363        1.0


mean
                          count       mean         std  nanmedian
TransactionDT_dayOfMonth                                         
1                         40309  30.891067  102.677442        4.0
2                         41482  31.043923  105.879018        3.0
3                         42315  32.279334  106.740637        4.0
4                         36785  30.121354  102.479329        3.0
5                         36239  30.615497  104.400912        3.0
6                         34901  29.084840  101.571394        3.0
7                         34025  30.405672  107.598524        3.0
8                         33712  28.870966  100.876444        3.0
9                         34167  30.676032  104.212712        3.0
10                        33906  30.996608  109.509896        3.0
11                        33075  33.173031  124.760036        3.0
12                        35238  32.927635  131.249348        3.0
13                        35054  41.020397  177.012605        3.0
14   

mean
                count      mean       std  nanmedian
DeviceInfo                                          
0PAJ5               1  1.000000       NaN        1.0
0PJA2               1  1.000000       NaN        1.0
0PM92               4  0.500000  0.577350        0.5
1016S               1  9.000000       NaN        9.0
2014819             1  1.000000       NaN        1.0
...               ...       ...       ...        ...
verykools5036       1  1.000000       NaN        1.0
verykools5702       1  1.000000       NaN        1.0
vivo                6  1.333333  0.516398        1.0
weimei_we           1  1.000000       NaN        1.0
xs-Z47b7VqTMxs      1  1.000000       NaN        1.0

[2799 rows x 4 columns]
std
                count      mean       std  nanmedian
DeviceInfo                                          
0PAJ5               1  1.000000       NaN        1.0
0PJA2               1  1.000000       NaN        1.0
0PM92               4  0.500000  0.577350        0.5
1016S       

In [7]:
train_test_transformed.head()

,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,...,smoothed_encoded_DeviceType_on_C13_nanmedian,smoothed_encoded_DeviceInfo_on_C13_mean,smoothed_encoded_DeviceInfo_on_C13_std,smoothed_encoded_DeviceInfo_on_C13_nanmedian,smoothed_encoded_OS_NAME_on_C13_mean,smoothed_encoded_OS_NAME_on_C13_std,smoothed_encoded_OS_NAME_on_C13_nanmedian,smoothed_encoded_id_30_on_C13_mean,smoothed_encoded_id_30_on_C13_std,smoothed_encoded_id_30_on_C13_nanmedian
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,1.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,2.0,0.0,1.0,0.0,25.0,1.0,5.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,16.458,61.951,2.053,13.724,46.208,1.002,3.234,26.554,1.006


In [24]:
def train_val_lgb():
    
    COLS_TO_USE = [
        i for i in train_test_transformed.columns
        if i not in COLUMNS_TO_REMOVE
    ]
    
    pars = {
        'num_leaves': 2 ** 8 - 1,
        'learning_rate': 0.05,
        'feature_fraction': 0.6 ,
        'bagging_fraction': 0.6,
        'bagging_freq': 51,
        'cat_smooth': 209,
        'lambda_l1': 1.0,
        'lambda_l2': 3.0,
        'max_bin': 270,
        'scale_pos_weight': 7.0,
        'boosting_type': 'gbdt',
        'objective': 'xentropy',
        'metric': ['auc'],
        'num_threads': -1
    }
    
    bound = 16
    
    
    COLS_TO_USE = [
        i for i in COLS_TO_USE 
        if not ('smoothed_encoded' in i and 'isFraud' in i) and not (i.startswith('card') and len(i) < 6)
    ]
    
    cat_c = [
        i for i in CATEGORICAL_FEATURES 
        if not ('smoothed_encoded' in i and 'isFraud' in i) and not (i.startswith('card') and len(i) < 6)
    ]
    
    COLS_TO_USE += ['smoothed_encoded_TransactionDT_hour_on_isFraud_mean',
'smoothed_encoded_TransactionDT_hour_on_isFraud_std',
'smoothed_encoded_TransactionDT_dayOfWeek_on_isFraud_mean',
'smoothed_encoded_TransactionDT_dayOfWeek_on_isFraud_std',
'smoothed_encoded_TransactionDT_dayOfMonth_on_isFraud_mean',
'smoothed_encoded_TransactionDT_dayOfMonth_on_isFraud_std',
'smoothed_encoded_TransactionDT_weekOfMonth_on_isFraud_mean',
'smoothed_encoded_TransactionDT_weekOfMonth_on_isFraud_std',
                    'smoothed_encoded_OS_NAME_on_isFraud_mean',
'smoothed_encoded_OS_NAME_on_isFraud_std',
            ]
    
    print(len(COLS_TO_USE))
    
    cur_train = train_test_transformed[
        train_test_transformed['TransactionDT_split'] <= bound
    ]
    
    cur_test = train_test_transformed[
        (train_test_transformed['TransactionDT_split'] > bound) & \
        (train_test_transformed['TransactionDT_split'] < 19)
    ]
    
    # create dataset for lightgbm
    lgb_train = lgb.Dataset(cur_train[COLS_TO_USE], cur_train['isFraud'])
    lgb_eval = lgb.Dataset(cur_test[COLS_TO_USE], cur_test['isFraud'], reference=lgb_train)
    
    gbm = lgb.train(
        pars,
        lgb_train,
        num_boost_round=20000,
        valid_sets=(lgb_train, lgb_eval),
        valid_names=('train', 'valid'),
        early_stopping_rounds=100,
        feature_name=COLS_TO_USE,
        categorical_feature=cat_c,
        verbose_eval=100
    )
    
    return gbm

In [25]:
%%time
lgb_fitted = train_val_lgb()

603
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.973757	valid's auc: 0.920576
[200]	train's auc: 0.991458	valid's auc: 0.927263
[300]	train's auc: 0.99672	valid's auc: 0.928692
[400]	train's auc: 0.998554	valid's auc: 0.929022
[500]	train's auc: 0.999226	valid's auc: 0.929345
Early stopping, best iteration is:
[423]	train's auc: 0.998834	valid's auc: 0.929593
CPU times: user 41min 23s, sys: 16.2 s, total: 41min 39s
Wall time: 3min 3s


In [17]:
for i in train_test_transformed.columns:
    
    if 'smoothed_encoded' in i and 'isFraud' in i and 'TransactionDT' in i:
        print(f"'{i}',")

'smoothed_encoded_TransactionDT_hour_on_isFraud_mean',
'smoothed_encoded_TransactionDT_hour_on_isFraud_std',
'smoothed_encoded_TransactionDT_dayOfWeek_on_isFraud_mean',
'smoothed_encoded_TransactionDT_dayOfWeek_on_isFraud_std',
'smoothed_encoded_TransactionDT_dayOfMonth_on_isFraud_mean',
'smoothed_encoded_TransactionDT_dayOfMonth_on_isFraud_std',
'smoothed_encoded_TransactionDT_weekOfMonth_on_isFraud_mean',
'smoothed_encoded_TransactionDT_weekOfMonth_on_isFraud_std',


In [38]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [49]:
pca_reducer = PCA(n_components=0.85)

In [50]:
%%time
V_fields = pca_reducer.fit_transform(
    StandardScaler()\
    .fit_transform(
        train_test_transformed[
            [i for i in train_test_transformed.columns if i.startswith('V')]
        ].fillna(0.0)
    )
)

CPU times: user 3min 50s, sys: 17.5 s, total: 4min 8s
Wall time: 36 s


In [51]:
V_fields

array([[-4.0115276 ,  0.88023451,  1.16512647, ..., -0.30192957,
        -0.16257286,  0.45072025],
       [-1.70104661, -0.09857212,  0.25199224, ..., -0.50058444,
        -0.05036983,  0.05172102],
       [-4.29355311,  0.7516503 , -0.88958054, ...,  0.12629188,
         0.04019963,  0.19611628],
       ...,
       [-4.29355311,  0.7516503 , -0.88958054, ...,  0.12629188,
         0.04019963,  0.19611628],
       [-4.29355311,  0.7516503 , -0.88958054, ...,  0.12629188,
         0.04019963,  0.19611628],
       [12.5924471 , -5.40463963, -5.23571458, ..., -0.53702687,
        -0.30155275,  0.31520839]])

In [52]:
V_fields.shape

(1097231, 37)